# CONNECT 4

## Informações Gerais

**Unidade Curricular:** Inteligência Artificial

**Turma:** PL6

**Grupo 2:**
- Hugo Duarte de Sousa — nº 202305900
- Isabela Britto Cartaxo — nº 202300339
- Tiago Figueredo Silva — nº 202000612

## Sumário

- [1. Introdução](#1-introducao)
- [2. Objetivos](#2-objetivos)
- [3. Lógica do Jogo e Estrutura do Tabuleiro](#3-logica-do-jogo-e-estrutura-do-tabuleiro)
- [4. Ciclo do Jogo e Modos de Interação](#4-ciclo-do-jogo-e-modos-de-interacao)


## 1. Introdução

Este trabalho foi desenvolvido no âmbito da unidade curricular de Inteligência Artificial (2024/2025) e tem como principal finalidade a aplicação de estratégias de procura adversária e algoritmos de aprendizagem supervisionada, com enfoque nas árvores de decisão. O projeto encontra-se dividido em duas componentes principais:

A implementação do algoritmo Monte Carlo Tree Search (MCTS) com a fórmula UCT, aplicado ao jogo Connect Four (Quatro em Linha), um clássico de estratégia entre dois jogadores.

A construção de um classificador baseado em árvores de decisão, recorrendo ao algoritmo ID3, treinado com dois conjuntos de dados distintos: o conhecido dataset Iris e um dataset gerado a partir de simulações com o MCTS no jogo Connect Four.

Este trabalho tem como intuito consolidar os conhecimentos teóricos abordados nas aulas, promovendo a sua aplicação prática em contextos desafiantes de tomada de decisão, planeamento e aprendizagem automática.

### O jogo Connect Four
O Connect Four (ou Quatro em Linha) é um jogo de estratégia para dois jogadores, jogado num tabuleiro vertical com 7 colunas e 6 linhas. Cada jogador tem 21 peças da sua cor (normalmente vermelho e amarelo), e o objetivo do jogo é ser o primeiro a alinhar quatro peças consecutivas na horizontal, vertical ou diagonal.

Os jogadores jogam alternadamente, inserindo uma peça numa das colunas. A peça cai até ocupar a posição mais baixa disponível nessa coluna. O jogo termina quando um dos jogadores vence ou quando o tabuleiro está cheio, resultando num empate.

## 2. Objetivos

- Implementar um agente capaz de jogar Connect Four utilizando o algoritmo MCTS com UCT.

- Suportar diferentes modos de jogo: humano vs. humano, humano vs. computador e computador vs. computador (com estratégias distintas).

- Implementar o algoritmo ID3 para gerar árvores de decisão a partir de dados, sem recorrer a bibliotecas externas como o scikit-learn.

- Avaliar a performance das árvores de decisão utilizando:

- O dataset Iris.

- Um dataset gerado artificialmente a partir da execução do MCTS no Connect Four, contendo pares (estado do jogo, jogada recomendada).

- Documentar as decisões tomadas, discutir os resultados obtidos.

## 3. Lógica do Jogo e Estrutura do Tabuleiro

A lógica do jogo Connect Four foi implementada através da classe Board, que representa o estado atual do tabuleiro, permite realizar jogadas, verificar jogadas legais e determinar se o jogo terminou com vitória ou empate.

### Estrutura do tabuleiro
O tabuleiro é representado por uma matriz de 6 linhas e 7 colunas (6x7), modelada como uma lista de listas de strings. Cada posição pode conter:

- "." – casa vazia

- "X" – peça do jogador 1

- "O" – peça do jogador 2

A matriz é inicializada com todas as casas vazias, e as peças "caem" até à posição mais baixa disponível em cada coluna, respeitando as regras do jogo real.

### Componentes principais da implementação
- self.board: Matriz 6x7 que guarda o estado atual do tabuleiro.

- self.y_coords: Dicionário que indica a próxima linha disponível em cada coluna, permitindo simular a gravidade.

- self.counter: Contador de jogadas realizadas (máximo 42).

- make_move(x, current_player): Realiza uma jogada válida, atualizando o tabuleiro e o estado interno.

- is_legal_move(x): Verifica se a jogada na coluna x é legal (coluna dentro dos limites e com espaço disponível).

- is_board_full(): Verifica se o tabuleiro está cheio.

- is_won(x, current_player): Verifica se a última jogada feita na coluna x resultou numa vitória para o jogador atual.

- is_tie(): Verifica se o jogo terminou empatado.

- get_possible_moves(current_player): Devolve todos os possíveis estados do tabuleiro resultantes de uma jogada válida do jogador atual (útil para o algoritmo MCTS).

In [ ]:
from copy import deepcopy

class Board:
    def __init__(self):
        """
        Initializes the Connect Four board.
        - 6 rows x 7 columns represented by a list of lists.
        - y_coords stores the next available row index for each column.
        """
        self.counter = 0
        self.board_width = 7
        self.board_height = 6
        self.board = [list(".......") for _ in range(self.board_height)]
        self.y_coords = {col: self.board_height - 1 for col in range(self.board_width)}
        self.last_move_column = None

    def to_tuple(self):
        """
        Returns an immutable representation of the board (useful for hashing).
        """
        return tuple(tuple(row) for row in self.board)

    def get_simulation_board(self):
        """
        Placeholder for compatibility or extensions (e.g., for neural network input).
        """
        return []

    def print_board(self):
        """
        Displays the board to the console.
        """
        print("\n" + " ".join(map(str, [1, 2, 3, 4, 5, 6, 7])))
        for row in self.board:
            print(" ".join(row))
        print()

    def is_empty(self, x, y) -> bool:
        """
        Checks if a given cell is empty.
        """
        return self.board[y][x] == "."

    def is_board_full(self) -> bool:
        """
        Checks if the board is full (42 moves).
        """
        return self.counter == 42

    def is_legal_move(self, x) -> bool:
        """
        Checks if a move is legal:
        - Column must be within bounds.
        - Column must not be full.
        """
        if self.is_board_full():
            return False
        if not 0 <= x < self.board_width:
            return False
        if self.y_coords[x] < 0:
            return False
        return self.is_empty(x, self.y_coords[x])

    def make_move(self, x, current_player):
        """
        Places a piece from the current player in the specified column.
        Updates the board, move counter, and last played column.
        """
        y = self.y_coords[x]
        self.board[y][x] = current_player
        self.counter += 1
        self.y_coords[x] -= 1
        self.last_move_column = x

    def count_in_direction(self, current_player, dx, dy, x, y):
        """
        Counts how many consecutive pieces exist in a given direction from (x, y).
        """
        count = 0
        while (0 <= x + dx < self.board_width and 
               0 <= y + dy < self.board_height and 
               self.board[y + dy][x + dx] == current_player):
            x += dx
            y += dy
            count += 1
        return count

    def is_won(self, x, current_player) -> bool:
        """
        Checks whether the last move in column x resulted in a win.
        """
        directions = [((0, 1), (0, -1)),      # Horizontal
                      ((1, 0), (-1, 0)),      # Vertical
                      ((1, 1), (-1, -1)),     # Main diagonal
                      ((1, -1), (-1, 1))]     # Anti-diagonal

        y = self.y_coords[x] + 1  # Find the row where the last move was placed
        if y is None:
            return False
        for (dy1, dx1), (dy2, dx2) in directions:
            total = (self.count_in_direction(current_player, dx1, dy1, x, y) +
                     self.count_in_direction(current_player, dx2, dy2, x, y) + 1)
            if total >= 4:
                return True
        return False

    def has_winner(self) -> bool:
        """
        Scans the board to detect if there is a winning sequence for any player.
        Used in end-of-game checks (e.g., for tie).
        """
        for y in range(self.board_height):
            for x in range(self.board_width):
                if self.board[y][x] in ["X", "O"]:
                    current_player = self.board[y][x]
                    for (dx1, dy1), (dx2, dy2) in [((0, 1), (0, -1)),
                                                  ((1, 0), (-1, 0)),
                                                  ((1, 1), (-1, -1)),
                                                  ((1, -1), (-1, 1))]:
                        count = 1
                        count += self.count_in_direction(current_player, dx1, dy1, x, y)
                        count += self.count_in_direction(current_player, dx2, dy2, x, y)
                        if count >= 4:
                            return True
        return False

    def is_tie(self) -> bool:
        """
        Returns True if the board is full and there is no winner.
        """
        return self.is_board_full() and not self.has_winner()

    def get_possible_moves(self, current_player):
        """
        Returns a list of possible future board states given the current player's move.
        Useful for MCTS simulation.
        """
        possible_boards = []
        for i in range(self.board_width):
            if self.is_legal_move(i):
                new_board = deepcopy(self)
                new_board.make_move(i, current_player)
                possible_boards.append(new_board)
        return possible_boards


## 4. Ciclo do Jogo e Modos de Interação

Nesta secção, implementamos o ciclo principal de jogo do Connect Four, incluindo os três modos de interação disponíveis:

- **Humano vs Humano**: dois jogadores humanos alternam jogadas, inserindo a coluna desejada. O programa valida se a jogada é legal e permite repetir em caso de erro. Também é possível pedir uma sugestão de jogada, gerada pela IA com base no algoritmo MCTS.

- **IA vs Humano**: o jogador humano enfrenta um agente inteligente baseado no algoritmo **Monte Carlo Tree Search (MCTS)**. A IA calcula a melhor jogada possível em cada turno. A jogabilidade é alternada, e o tempo de execução da decisão da IA é exibido ao utilizador.

- **IA vs IA**: duas instâncias da IA jogam entre si, cada uma utilizando MCTS para escolher as suas jogadas. O jogo corre de forma automática e os tempos de decisão são registados e apresentados.

Além dos modos de jogo, também estão disponíveis funções para **geração de datasets**, que simulam partidas completas para fins de treino de modelos de aprendizagem supervisionada, nomeadamente árvores de decisão com o algoritmo ID3:

- **ai_vs_ai_simulation_generator()**: simula jogos entre IAs, gravando o estado do tabuleiro, o jogador da vez e a jogada ótima sugerida.

### Lógica do ciclo de jogo

O ciclo de jogo é executado num laço while que termina quando:
- o tabuleiro está cheio (is_board_full()), ou
- um jogador vence (is_won()), ou
- um empate é detetado (is_tie()).

Em cada iteração:
1. O estado atual do tabuleiro é impresso (print_board()).
2. O jogador da vez é definido.
3. Dependendo do modo, a jogada é obtida via input humano ou decisão da IA.
4. A jogada é validada e executada.
5. O jogo verifica se terminou.

### Tratamento de erros e robustez

Para garantir uma experiência de jogo mais segura e intuitiva, foram implementadas verificações rigorosas de input nos momentos críticos da interação com o utilizador:

- Na escolha do modo de jogo, o sistema rejeita qualquer valor inválido (como letras, símbolos ou números fora do intervalo 1–3), solicitando ao utilizador que introduza uma opção válida.

- Durante as jogadas humanas, o programa valida se a entrada corresponde a um número entre 1 e 7. Caso contrário, uma mensagem de erro é exibida e o jogador é convidado a tentar novamente, sem perder a vez.

- O programa também impede jogadas em colunas já cheias, apresentando um aviso e aguardando uma nova escolha válida.

### Execução

Este é o menu principal do programa. Existem três modos de jogo distintos que podem ser seleccionados ao executar a função run().  
No entanto, como o Jupyter Notebook **não permite input interativo direto**, para demonstrar o funcionamento do algoritmo MCTS será utilizada a função ai_vs_ai(x_simulation_limit, o_simulation_limit), que executa uma partida automática entre duas IAs.


In [ ]:
import time
import numpy as np
from copy import deepcopy

def human_play(game, current_player):
    """
    Handles human player's input. Allows hint request or direct column input.
    Ensures only valid numeric input in the range 1–7.
    """
    while True:
        print("Make a move by choosing your coordinates to play.")
        print("Enter column (1-7) or type 'hint' for a suggestion:")
        user_input = input()

        if user_input.lower() == 'hint':
            best_move = get_hint(game, current_player)
            print("Best move according to the AI: ", best_move)
            continue  # Ask again after showing the hint

        if not user_input.isdigit():
            print("Invalid input. Please enter a number between 1 and 7.")
            continue

        move = int(user_input) - 1
        if 0 <= move < 7:
            return move
        else:
            print("Column out of range. Try again.")


def get_hint(game, current_player):
    """
    Uses MCTS to provide a hint for the current player.
    """
    root = Node(game, None)
    mcts = MCTS(root, current_player)
    best_node = mcts.best_move()
    return best_node.board.last_move_column + 1


def human_vs_human():
    """
    Human vs. Human game loop.
    """
    game = Board()
    player = "X"
    game_over = False

    while not game.is_board_full() and not game_over:
        player = "O" if player == "X" else "X"
        game.print_board()
        print(f"It is now {player}'s turn!")

        valid_move = False
        while not valid_move:
            col = human_play(game, player)
            if not game.is_legal_move(col):
                print("\nWarning: Invalid move. Try again!")
            else:
                game.make_move(col, player)
                valid_move = True

        game_over = game.is_won(col, player)

    game.print_board()
    print("It's a tie!" if game.is_tie() else f"Player {player} has won!\n")


def ai_vs_human():
    """
    AI vs. Human game loop.
    """
    game = Board()
    ai = "O"
    human = "X"
    human_turn = False
    game_over = False

    while not game.is_board_full() and not game_over:
        current_player = human if human_turn else ai
        game.print_board()
        print(f"It is now {current_player}'s turn!")

        if human_turn:
            valid_move = False
            while not valid_move:
                move = human_play(game, current_player)
                if not game.is_legal_move(move):
                    print("\nWarning: Invalid move. Try again!")
                else:
                    game.make_move(move, current_player)
                    valid_move = True
        else:
            root = Node(game, None)
            mcts = MCTS(root, ai)
            start = time.time()
            best_node = mcts.best_move()
            move = best_node.board.last_move_column
            game.make_move(move, ai)
            end = time.time()
            print(f"AI chose column: {move + 1}\nTime taken: {end - start:.2f}s")

        game_over = game.is_won(move, current_player)
        human_turn = not human_turn

    game.print_board()
    print("It's a tie!" if game.is_tie() else f"Player {current_player} has won!\n")


def ai_vs_ai(x_simulation_limit=10000, o_simulation_limit=10000):
    """
    AI vs. AI game loop.
    """
    game = Board()
    players = ["X", "O"]
    current_index = 0
    game_over = False

    while not game.is_board_full() and not game_over:
        current_player = players[current_index]
        game.print_board()
        print(f"It is now {current_player}'s turn!")

        root = Node(game, None)
        mcts = MCTS(root, current_player)
        start = time.time()
        best_node = mcts.best_move()
        move = best_node.board.last_move_column
        game.make_move(move, current_player)
        print(move)
        end = time.time()

        print(f"{current_player} chose column: {move + 1}\nTime taken: {end - start:.2f}s")
        game_over = game.is_won(move, current_player)
        current_index = 1 - current_index

    game.print_board()
    print("It's a tie!" if game.is_tie() else f"Player {current_player} has won!\n")


def run():
    """
    Entry point to choose game mode. Validates input.
    """
    print("Choose a game mode:")
    print("1. Human vs Human")
    print("2. AI vs Human")
    print("3. AI vs AI")

    while True:
        user_input = input("Enter the game mode number (1-3): ")
        if user_input.isdigit():
            game_mode = int(user_input)
            if game_mode in [1, 2, 3]:
                break
        print("Invalid game mode. Please enter 1, 2 or 3.")

    if game_mode == 1:
        human_vs_human()
    elif game_mode == 2:
        ai_vs_human()
    elif game_mode == 3:
        ai_vs_ai()


def ai_vs_ai(x_simulation_limit=10000, o_simulation_limit=10000):
    """
    Simulates AI vs AI match to generate a dataset of board states, player turns and optimal moves.
    """
    game = Board()
    players = ["X", "O"]
    current_index = 0
    game_over = False

    while not game.is_board_full() and not game_over:
        current_player = players[current_index]
        game.print_board()
        print(f"It is now {current_player}'s turn!")

        sim_limit = x_simulation_limit if current_player == "X" else o_simulation_limit
        root = Node(game, None)
        mcts = MCTS(root, current_player, sim_limit)
        start = time.time()
        best_node = mcts.best_move()
        move = best_node.board.last_move_column
        game.make_move(move, current_player)
        end = time.time()

        print(f"{current_player} chose column: {move + 1}\nTime taken: {end - start:.2f}s")
        game_over = game.is_won(move, current_player)
        current_index = 1 - current_index

    game.print_board()
    print("It's a tie!" if game.is_tie() else f"Player {current_player} has won!\n")



## 5. Monte Carlo Tree Search

Nesta secção, apresentamos a implementação do algoritmo **Monte Carlo Tree Search (MCTS)**, utilizado como estratégia de decisão para o jogo Connect Four.

O MCTS baseia-se em simulações aleatórias para explorar o espaço de estados do jogo e selecionar as jogadas mais promissoras. A cada iteração, o algoritmo executa quatro fases principais:

1. **Seleção**: navega pela árvore de decisão até um nó que ainda não esteja totalmente expandido.
2. **Expansão**: adiciona um novo nó filho a partir de uma jogada possível ainda não explorada.
3. **Simulação**: executa uma partida aleatória a partir do novo estado até ao final do jogo.
4. **Retropropagação**: propaga o resultado da simulação para os nós ancestrais, atualizando os valores de vitória e visita.


### Estrutura do Nó (Node)

A classe Node representa um estado na árvore de MCTS. Cada nó contém o estado atual do tabuleiro, uma ligação ao nó pai, os seus filhos, e estatísticas que guiam o algoritmo de seleção.


In [3]:
import math

class Node:
    def __init__(self, board, parent=None):
        """
        Initializes a node representing a game state in the MCTS tree.

        Parameters:
        - board: The game board associated with this node.
        - parent: The parent node in the tree (None if this is the root).
        """
        self.board = board
        self.parent = parent
        self.children = []
        self.wins = 0
        self.visits = 0

    def is_fully_expanded(self) -> bool:
        """
        Returns True if all possible moves from this state have been expanded into children.
        """
        return len(self.children) == len(self.board.get_possible_moves("X"))  # or "O" if alternating

    def best_child(self):
        """
        Returns the child node with the highest UCT (Upper Confidence Bound for Trees) value.
        Used to select the most promising node during the selection phase of MCTS.
        """
        return max(self.children, key=lambda child: child.get_uct_value())

    def add_child(self, child):
        """
        Adds a new child node to this node.
        """
        self.children.append(child)

    def get_uct_value(self, c=1.4) -> float:
        """
        Calculates the UCT value for this node, balancing exploration and exploitation.
        """
        exploitation = self.wins / (self.visits + 1e-6)
        exploration = c * math.sqrt(math.log(self.parent.visits + 1) / (self.visits + 1e-6))
        return exploitation + exploration


### Estrutura do MCTS

A classe MCTS encapsula toda a lógica do algoritmo. Abaixo, destacamos os principais métodos implementados:

- selection()  
  Seleciona o melhor caminho atual com base nos valores de UCT.

- expansion(node)  
  Cria todos os filhos possíveis para um dado nó, representando as próximas jogadas válidas.

- simulation(node)  
  Executa uma simulação aleatória até o fim do jogo, retornando o vencedor ou empate.

- backpropagation(node, result)  
  Atualiza estatísticas (vitórias e visitas) com base no resultado da simulação.

- check_for_win(leaf)  
  Verifica se existe uma jogada imediata que leva à vitória — se existir, executa essa jogada diretamente (acelera o processo).

- best_move()  
  Executa o ciclo completo do MCTS (seleção, expansão, simulação, retropropagação) um número de vezes definido por simulation_limit, e retorna o melhor nó filho da raiz.


In [4]:
import random
from copy import deepcopy
from Node import Node

class MCTS:
    def __init__(self, initial_state, current_player, simulation_limit=10000):
        """
        Initializes the MCTS agent.

        Parameters:
        - initial_state: The root node containing the current game state.
        - current_player: The player for whom the move is being calculated.
        - simulation_limit: Maximum number of simulations to run (default: 10000).
        """
        self.root = initial_state
        self.simulation_limit = simulation_limit
        self.current_player = current_player
        self.gameOver = False

    def selection(self):
        """
        Selects the most promising node by traversing the tree
        using the UCT value until a leaf is reached.
        """
        node = self.root
        while node.children:
            node = node.best_child()
        return node

    def expansion(self, node):
        """
        Expands the given node by generating all possible child states.
        """
        current_player = self.current_player
        for board in node.board.get_possible_moves(current_player):
            new_node = Node(board, parent=node)
            node.add_child(new_node)

    def simulation(self, node):
        """
        Simulates a random playout from the current node until
        the game ends with a win or a tie.
        """
        sim_board = deepcopy(node.board)
        player = "O" if self.current_player == "X" else "X"

        while not sim_board.is_board_full():
            legal_moves = [i for i in range(sim_board.board_width) if sim_board.is_legal_move(i)]
            move = random.choice(legal_moves)
            sim_board.make_move(move, player)

            if sim_board.is_won(move, player):
                return player

            player = "O" if player == "X" else "X"

        return "."

    def backpropagation(self, node, result):
        """
        Updates the statistics of the nodes on the path from the
        simulation result back to the root.
        """
        while node is not None:
            node.visits += 1
            if result == self.current_player:
                node.wins += 1
            elif result == ".":
                node.wins += 0.5
            node = node.parent

    def check_for_win(self, leaf) -> bool:
        """
        Checks if there is an immediate winning move for the current player.
        If found, the move is made directly.
        """
        possible_moves = [i for i in range(leaf.board.board_width) if leaf.board.is_legal_move(i)]

        for i in possible_moves:
            simulated_node = deepcopy(leaf)
            simulated_node.board.make_move(i, self.current_player)
            if simulated_node.board.is_won(i, self.current_player):
                leaf.board.make_move(i, self.current_player)
                self.gameOver = True
                return True

        return False

    def best_move(self):
        """
        Runs the full MCTS process and returns the best child of the root node.
        """
        leaf = self.selection()
        if not leaf.children:
            self.expansion(leaf)

        if self.check_for_win(leaf):
            return leaf

        for child in leaf.children:
            result = self.simulation(child)
            self.backpropagation(child, result)

        children = leaf.children

        for _ in range(7, self.simulation_limit + 1):
            selected_leaf = random.choice(children)
            result = self.simulation(selected_leaf)
            self.backpropagation(selected_leaf, result)

        return self.root.best_child()


### Execução da simulação

Como o Jupyter Notebook **não permite interações com `input()` em tempo real**, os modos de jogo que requerem ações do utilizador (como Humano vs IA) **não podem ser executados diretamente aqui**.

No entanto, é possível correr simulações automáticas entre duas IAs utilizando a função `ai_vs_ai()`. Nesta função, o número de simulações por jogada pode ser ajustado através do parâmetro `simulation_limit`, permitindo observar o impacto da profundidade da busca no desempenho da IA.

Sinta-se à vontade para experimentar diferentes valores de `simulation_limit` para comparar os tempos de execução e a qualidade das decisões.

In [8]:
ai_vs_ai(x_simulation_limit=10000, o_simulation_limit=10000)


1 2 3 4 5 6 7
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .

It is now X's turn!
X chose column: 4
Time taken: 1.91s

1 2 3 4 5 6 7
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . X . . .

It is now O's turn!
O chose column: 3
Time taken: 1.79s

1 2 3 4 5 6 7
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . O X . . .

It is now X's turn!
X chose column: 4
Time taken: 1.73s

1 2 3 4 5 6 7
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . X . . .
. . O X . . .

It is now O's turn!
O chose column: 4
Time taken: 1.67s

1 2 3 4 5 6 7
. . . . . . .
. . . . . . .
. . . . . . .
. . . O . . .
. . . X . . .
. . O X . . .

It is now X's turn!
X chose column: 3
Time taken: 1.69s

1 2 3 4 5 6 7
. . . . . . .
. . . . . . .
. . . . . . .
. . . O . . .
. . X X . . .
. . O X . . .

It is now O's turn!
O chose column: 4
Time taken: 1.60s

1 2 3 4 5 6 7
. . . . . . .
. . . . . . .
. . . O . . .
. . . O

# Creating the Dataset

300 games are recorded using MCTS to generate the best move with 5000 simulations per move


The dataset has 44 columns, the first 42 are the squares of the board, 43rd column contains the current player, in other words, whose to play, and the 44th column is the column with the supposed best move generated by the MCTS algorithm



*ai_vs_ai_simulotion_generator()*
- An adaptation of the ai_vs_ai() code;
- Returns 3 differnet arrays that stores the boards, player turns and optimal moves after each game;

*generate_db(iteration, last_row)*
- Iteration: number of games to iterate
- Last_row: last row written in Excel so the user can return to writing in the file without resetting or overwriting any previous data

#### It is normal for error to occur, since the file name is not specified

In [8]:
import numpy as np
import openpyxl 


def ai_vs_ai_simulation_generator():
    game = Board()
    boardStates = [] # 2D arrays
    playerTurns = []  # strings
    optimalMoves = [] # integers
    x = "X"
    o = "O"
    playerX = True # False for O first, True for X first

    game_is_over = False

    while (not game.is_board_full() and not game_is_over) or game.is_tie():
        currentplayer = x if playerX else o
        game.print_board()

        if playerX:
            root = Node(game, None)
            mcts = MCTS(root, x, 5000)
            best_node = mcts.best_move()
            move = best_node.board.last_move_column   
            if mcts.gameOver:
                break
    
        else:
            root = Node(game, None)
            mcts = MCTS(root, o, 5000)
            best_node = mcts.best_move()
            move = best_node.board.last_move_column
            if mcts.gameOver:
                break
        
        currBoard = np.array(game.board).flatten().tolist()  # Turns the current board into a 1D array

        boardStates.append(currBoard) # Adding the 1D array to the all the boards played
        playerTurns.append(currentplayer)
        optimalMoves.append(move + 1)

        game.make_move(move, currentplayer)
        playerX = False if playerX else True

        print(f"{currentplayer} chose column :", move + 1)

    # After game is won
    game.print_board()

    if game.is_tie():
        print("It's a tie!")
        return boardStates, playerTurns, optimalMoves
    
    else:
        print(f"Player {currentplayer} has won!\n")
        return boardStates, playerTurns, optimalMoves
    


file = "FILE_NAME.xlsx"
workbook = openpyxl.load_workbook(file)
worksheet = workbook.active



def run_simulation():
    boardStates, playerTurns, optimalMoves = ai_vs_ai_simulation_generator()   # RETURNS 3 lists, all the baords across the simulated game, player turns and respective optimal move
    
    return boardStates, playerTurns, optimalMoves


def generate_db(iterations=150, last_row=1):  # last_row defualt = 1, if last row written in the dataset is n, set last_row = n
    last_updated_row = 0
    for i in range(iterations):
        print(f"Initiating simulation {i + 1}:")
        boardStates, playerTurns, optimalMoves = run_simulation()

        print("Writting Data...")
        # Since each of the board size varries,
        allocate = len(boardStates)
        for j in range(allocate):
            for k in range(1, 43):
                worksheet.cell(row=j + 1 + last_row + last_updated_row, column=k, value=f"{boardStates[j][k - 1]}")
            worksheet.cell(row=j + 1 + last_row + last_updated_row, column=43, value=f"{playerTurns[j]}")
            worksheet.cell(row=j + 1 + last_row + last_updated_row, column=44, value=optimalMoves[j])
            workbook.save("FILE_NAME.XLSX")

        last_updated_row += allocate




        print(f"simulation {i + 1} has completed!")
        print("Upload completed")
        print(f"Progression { round(((i + 1)/iterations) * 100, 2)}%")
        print("")


generate_db()

FileNotFoundError: [Errno 2] No such file or directory: 'FILE_NAME.xlsx'

# Decision Tree

### Node class

In [10]:
class Node_DT:

    def __init__(self, feature = None, split_criteria = None, left = None, right = None, value=None):
        self.feature = feature                      # Ex.: numero de quartos
        self.split_criteria = split_criteria        # Ex.: <= 5 do lado esquerdo; > 5 do lado direito ==> split_criteria = 5
        self.left = left                            # nó da esquerda
        self.right = right                          # nó da direita
        self.value = value                          # caso seja uma folha, qual o valor
        self.children = {}                          # Para guardar os branches

    # metodo para avaliar se é uma folha não
    def is_leaf(self) -> bool:
        return self.value is not None

### Decision Tree class

In [11]:
from collections import Counter

class DecisinoTree:

    def __init__(self, min_sample_split=2, max_depth= 15, n_features=None) -> None:
        self.min_sample_split = min_sample_split    # Mínimo de amostras necessário para dividir um nó.
        self.max_depth = max_depth                  # Profundidade máxima da árvore.
        self.n_features = n_features                # Número de características usadas para realizar os splits.
        self.root = None
        
    def fit(self, X, y):
        self.n_features = X.shape[1]
        self.root = self.grow_tree(X, y)

    def grow_tree(self, X, y, depth=0):
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))
        
        if (depth >= self.max_depth or n_labels == 1 or n_samples < self.min_sample_split):
            leaf_value = self.most_common_label(y)
            return Node_DT(value=leaf_value)
        
        best_feature, best_value = self.best_split(X, y, list(range(n_feats)))
        
        # Create node with best feature
        node = Node_DT(feature=best_feature)
        
        # For each unique value in the feature
        for value in np.unique(X.iloc[:, best_feature]):
            # Get indices for this value
            idxs = self.split(X.iloc[:, best_feature], value)
            if len(idxs) > 0:
                # Recursively grow subtree
                node.children[value] = self.grow_tree(
                    X.iloc[idxs, :], 
                    y.iloc[idxs], 
                    depth + 1
                )
        
        return node


    def most_common_label(self, y):
        counter = Counter(y)
        value = counter.most_common(1)
        if value:
            return value[0][0]
        return value


    # This is fine 
    def best_split(self, X, y, feat_idxs): # $
        if feat_idxs is None:
            feat_idxs = range(X.shape[1])
        best_gain = -1
        best_split_feat, best_split_value = None, None

        for feat_idx in feat_idxs:
            X_column = X.iloc[:, feat_idx]              # Selects all the rows with the spesified index, ou seja, this returns a series
            limits = np.unique(X_column)                # A list of [. X O ]

            for split_criteria in limits:               # split_criteria = . or x or o
                # calculate the information gain
                gain = self.information_gain(y, X_column, split_criteria)

                if gain > best_gain:
                    best_gain = gain
                    best_split_feat = feat_idx
                    best_split_value = split_criteria

        return best_split_feat, best_split_value



    def information_gain(self, y, X_column, split_criteria):
        parent_entropy = self.entropy(y)
        matching_idxs = self.split(X_column, split_criteria)
        non_matching_idxs = np.setdiff1d(np.arange(len(X_column)), matching_idxs)

        if len(matching_idxs) == 0 or len(non_matching_idxs) == 0:
            return 0

        n = len(y)
        n_matching = len(matching_idxs)
        n_non_matching = len(non_matching_idxs)

        e_matching = self.entropy(y.iloc[matching_idxs])
        e_non_matching = self.entropy(y.iloc[non_matching_idxs])

        child_entropy = (n_matching/n) * e_matching + (n_non_matching/n) * e_non_matching

        information_gain = parent_entropy - child_entropy

        return max(0, information_gain)


    def split(self, X_column, value):
        # For categorical data, we want exact matches
        matching_idxs = np.argwhere(X_column == value).flatten()
        return matching_idxs


    # Entropy look correct 
    def entropy(self, y): # $
        counts = np.bincount(y)
        ps = counts / len(y) 
        return - np.sum([p * np.log2(p) for p in ps if p > 0])


    def predict(self, X):
        return list(self.traverse_tree(x, self.root) for x in X.values)


    def traverse_tree(self, x, node):
        if node.is_leaf():
            return node.value
        
        if x[node.feature] in node.children:
            return self.traverse_tree(x, node.children[x[node.feature]])
        return node.value
    


def accuracy(y_test, y_pred):
    return np.mean(y_test == y_pred) * 100

- grow_tree(): This is the main loop of the program, it will recursively grow out the tree ...
- best_split(): Iterates through all the features available and returns the best possible split;
- information_gain(): return the information gain of the subset;
- split(): auxilary function that splits the set into subset based on the chosen feature;
- entropy(): auxilary function that retuns the entropy of the subset;
- predict(): auxilary function that traverses the tree;
- traverse_tree(): recursive function that takes the test cases as input and returns the prediction;


#### More information:

The best split is determined by iterating through all the possible columns and the unique values of each column. For each feature have at most 3 different values, the data is split based on whether the feature's value matches the given value. Information gain is then calculated by computing the entropy of the resulting splits, those that match the value and those that don’t. The best split is the feature that gives the highest information gain.

## Data preprocessing

The tree is trained with a 80% training and 20% testing

In [12]:
import pandas as pd

df = pd.read_csv("Connect4-dataset.csv")

# optinal shuffle for randomness

x = df.drop("Best_Move", axis=1)
y = df["Best_Move"]

# get the dataset dimensions
number_of_rows = df.shape[0]
number_of_cols = df.shape[1]

cutoff = int(0.8 * number_of_rows)

# Training set
X_train = x.iloc[:cutoff, :]
Y_train = y.iloc[:cutoff]

# Testing set
X_test = x.iloc[cutoff:, :]
Y_test = y.iloc[cutoff:]

## Cross validation

### Fold 1

In [63]:
import pandas as pd

df = pd.read_csv("Connect4-dataset.csv")

# optinal shuffle for randomness

x = df.drop("Best_Move", axis=1)
y = df["Best_Move"]

fold1 = int(0.2 * number_of_rows)

X_train = x.iloc[fold1:, :]
Y_train = y.iloc[fold1:]

X_test = x.iloc[:fold1, :]
Y_test = y.iloc[:fold1]

### Fold 2

In [60]:
import pandas as pd

df = pd.read_csv("Connect4-dataset.csv")

x = df.drop("Best_Move", axis=1)
y = df["Best_Move"]

k = 5
n = len(df)
fold_size = n // k

# index starting from 0
fold_idx = 1 

start = fold_idx * fold_size
end = (fold_idx + 1) * fold_size if fold_idx < k - 1 else n  # handle last fold

# I need the data staerting from n and ending at 2n 

X_train = pd.concat([x.iloc[:start], x.iloc[end:]])
Y_train = pd.concat([y.iloc[:start], y.iloc[end:]])

X_test = x.iloc[start:end]
Y_test = y.iloc[start:end]

### Fold 3

In [49]:
import pandas as pd

df = pd.read_csv("Connect4-dataset.csv")

x = df.drop("Best_Move", axis=1)
y = df["Best_Move"]

k = 5
n = len(df)
fold_size = n // k

# index starting from 0
fold_idx = 2 

start = fold_idx * fold_size
end = (fold_idx + 1) * fold_size if fold_idx < k - 1 else n  # handle last fold

# I need the data staerting from n and ending at 2n 

X_train = pd.concat([x.iloc[:start], x.iloc[end:]])
Y_train = pd.concat([y.iloc[:start], y.iloc[end:]])

X_test = x.iloc[start:end]
Y_test = y.iloc[start:end]

### Fold 4


In [66]:
import pandas as pd

df = pd.read_csv("Connect4-dataset.csv")

x = df.drop("Best_Move", axis=1)
y = df["Best_Move"]

k = 5
n = len(df)
fold_size = n // k

# index starting from 0
fold_idx = 3  

start = fold_idx * fold_size
end = (fold_idx + 1) * fold_size if fold_idx < k - 1 else n  # handle last fold

# I need the data staerting from n and ending at 2n 

X_train = pd.concat([x.iloc[:start], x.iloc[end:]])
Y_train = pd.concat([y.iloc[:start], y.iloc[end:]])

X_test = x.iloc[start:end]
Y_test = y.iloc[start:end]

### Fold 5

In [67]:
import pandas as pd

df = pd.read_csv("Connect4-dataset.csv")

x = df.drop("Best_Move", axis=1)
y = df["Best_Move"]

k = 5
n = len(df)
fold_size = n // k

# index starting from 0
fold_idx = 4  

start = fold_idx * fold_size
end = (fold_idx + 1) * fold_size if fold_idx < k - 1 else n  # handle last fold

# I need the data staerting from n and ending at 2n 

X_train = pd.concat([x.iloc[:start], x.iloc[end:]])
Y_train = pd.concat([y.iloc[:start], y.iloc[end:]])

X_test = x.iloc[start:end]
Y_test = y.iloc[start:end]

### Train Tree

In [68]:
tree = DecisinoTree()
tree.fit(X_train, Y_train)
prediction = tree.predict(X_test)

print("Tree constructed successfully!")

Tree constructed successfully!


### Accuracy

In [69]:

acc = accuracy(Y_test, prediction)

print("")
print(f"Model accuracy: {round(acc, 2)}%")
print("")


Model accuracy: 50.97%



# **Conclusão**

In this project, we built a Connect 4 game from scratch in Python. Throughout the process, we explored ways to optimize game logic and performance, implemented the Monte Carlo Tree Search (MCTS) algorithm, and developed an intuitive algorithm interface. Additionally, we also learnt how to creat a dataset and built a decision tree model that is easy to interpret.

We are particularly proud of our MCTS implementation, which is capable of running up to 20000 simulations in just a few seconds. The game supports three game modes: Human vs Human, Human vs AI, and AI vs AI, with adjustable AI difficulty by tuning the number of MCTS iterations.

The decision tree model performed better than initially expected. Although we started with an accuracy of 47%, after experimented with various split criteria and configurations, the model achieved an improved accuracy of 50.97%.

To further validate the model, we used 5-fold cross-validation, which revealed variations in accuracy performance across different folds:
- Fold 1: 52.27%
- Fold 2: 54.11%
- Fold 3: 47.02%
- Fold 4: 49.68%
- Fold 5: 50.97%

These results demonstrate that cross-validation helped us uncover the potential of our decision tree beyond the initial test set.
